In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from tqdm import tqdm
import random
import os
import keras 

%matplotlib inline

from keras_radam import RAdam
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
np.random.seed(7)
random.seed(7)
#tf.random.set_seed(7)

In [3]:
train_ = pd.read_csv('./data/train.csv',index_col='id')
test_ = pd. read_csv('./data/test.csv',index_col='id')

In [4]:
train=train_.copy()
test=test_.copy()

In [5]:
# 기상청 데이터만 추출
x_train = train.loc[:,'X00':'X39']

# standardization을 위해 평균과 표준편차 구하기
MEAN = x_train.mean()
STD = x_train.std()

# 표준편차가 0일 경우 대비하여 1e-07 추가 
x_train = (x_train - MEAN) / (STD + 1e-07)

In [6]:
# RNN 모델에 입력 할 수 있는 시계열 형태로 데이터 변환 
def convert_to_timeseries(df, interval):
    sequence_list = []
    target_list = []
    
    for i in tqdm(range(df.shape[0] - interval)):
        sequence_list.append(np.array(df.iloc[i:i+interval,:-1]))
        target_list.append(df.iloc[i+interval,-1])
    
    sequence = np.array(sequence_list)
    target = np.array(target_list)
    
    return sequence, target

In [7]:
y_columns = ['Y15','Y16']
#y_columns= train.iloc[:,40:-1].columns

In [8]:
n_inte = 24

In [9]:
# t시점 이전 120분의 데이터로 t시점의 온도를 추정할 수 있는 학습데이터 형성
sequence = np.empty((0, n_inte, 40))
target = np.empty((0,))

for column in y_columns :
    
    concat = pd.concat([x_train, train[column]], axis = 1)

    _sequence, _target = convert_to_timeseries(concat.head(144*30), interval = n_inte)

    sequence = np.vstack((sequence, _sequence))
    target = np.hstack((target, _target))

100%|████████████████████████████████████████████████████████████████████████████| 4296/4296 [00:01<00:00, 2789.70it/s]


In [10]:
sequence.shape

(8592, 24, 40)

In [11]:
# convert_to_timeseries 함수를 쓰기 위한 dummy feature 생성
x_train['dummy'] = 0

In [12]:
# train set에서 도출된 평균과 표준편차로 standardization 실시 
test = (test - MEAN) / (STD + 1e-07)

In [13]:
# convert_to_timeseries 함수를 쓰기 위한 dummy feature 생성
test['dummy'] = 0

In [14]:
# train과 test 기간을 합쳐서 120분 간격으로 학습데이터 재구축
x_test, _ = convert_to_timeseries(pd.concat([x_train, test], axis = 0), interval=n_inte)

100%|██████████████████████████████████████████████████████████████████████████| 16248/16248 [00:06<00:00, 2609.76it/s]


In [15]:
# test set 기간인 후반부 80일에 맞게 자르기 
x_test = x_test[-11520:, :, :]

In [16]:
# 만들어 두었던 dummy feature 제거
x_train.drop('dummy', axis = 1, inplace = True)
test.drop('dummy', axis = 1, inplace = True)

In [17]:
# 간단한 lstm 모델 구축하기 
simple_lstm_model = keras.models.Sequential([
    keras.layers.LSTM(32, input_shape=sequence.shape[-2:]),
    keras.layers.Dense(64, activation='elu'),
    keras.layers.Dense(32, activation='elu'),    
    keras.layers.Dense(1)
])

simple_lstm_model.compile(optimizer='adam', loss='mse')

Instructions for updating:
Colocations handled automatically by placer.


In [18]:
# loss가 4미만으로 떨어지면 학습 종료 시키는 기능
class myCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = None):
        if(logs.get('loss') < 0.0001):
            print('\n Loss is under 0.0001, cancelling training')
            self.model.stop_training = True

In [19]:
callbacks = myCallback()

In [20]:
# # 간단한 lstm 모델 구축하기 
# simple_lstm_model = keras.models.Sequential([
#     keras.layers.LSTM(64, input_shape=sequence.shape[-2:]),
#     keras.layers.Dense(64, activation='elu'),
#     keras.layers.Dense(32, activation='elu'),    
#     keras.layers.Dense(1)
# ])

# simple_lstm_model.compile(optimizer='adam', loss='mse')
# # 모델 학습
# simple_lstm_model.fit(sequence, target, epochs=60, batch_size=128, verbose=2,
#     shuffle=False, callbacks = [callbacks], validation_split=0.1)

In [21]:
# 간단한 lstm 모델 구축하기 
simple_lstm_model = keras.models.Sequential([
    keras.layers.LSTM(32, input_shape=sequence.shape[-2:]),
#    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='elu'),
    keras.layers.Dense(1)
])
simple_lstm_model.compile(optimizer=Adam(lr=0.001/2.), loss='mse')
# 모델 학습
simple_lstm_model.fit(sequence, target, epochs=80, batch_size=128, verbose=2,
    shuffle=False, callbacks = [callbacks], validation_split=0.1)

# Epoch 80/80
#  - 1s - loss: 2.5424 - val_loss: 2.6561

Instructions for updating:
Use tf.cast instead.
Train on 7732 samples, validate on 860 samples
Epoch 1/80
 - 2s - loss: 515.9444 - val_loss: 546.9694
Epoch 2/80
 - 1s - loss: 402.4701 - val_loss: 385.5063
Epoch 3/80
 - 1s - loss: 243.8521 - val_loss: 202.0478
Epoch 4/80
 - 1s - loss: 129.7197 - val_loss: 85.1263
Epoch 5/80
 - 1s - loss: 71.9710 - val_loss: 44.9100
Epoch 6/80
 - 1s - loss: 46.5489 - val_loss: 29.9834
Epoch 7/80
 - 1s - loss: 30.2111 - val_loss: 21.1113
Epoch 8/80
 - 1s - loss: 21.9895 - val_loss: 16.1457
Epoch 9/80
 - 1s - loss: 16.8409 - val_loss: 13.3638
Epoch 10/80
 - 1s - loss: 13.0574 - val_loss: 10.6955
Epoch 11/80
 - 1s - loss: 10.8491 - val_loss: 9.1543
Epoch 12/80
 - 1s - loss: 9.4456 - val_loss: 8.3714
Epoch 13/80
 - 1s - loss: 8.5378 - val_loss: 7.7496
Epoch 14/80
 - 1s - loss: 7.8466 - val_loss: 7.2473
Epoch 15/80
 - 1s - loss: 7.4046 - val_loss: 6.8924
Epoch 16/80
 - 1s - loss: 7.0202 - val_loss: 6.6629
Epoch 17/80
 - 1s - loss: 6.7110 - val_loss: 6.4416
Ep

In [22]:
# # 간단한 lstm 모델 구축하기 
# simple_lstm_model = keras.models.Sequential([
#     keras.layers.LSTM(64, input_shape=sequence.shape[-2:], activation='tanh'),
#     keras.layers.Dense(64, activation='elu'),
#     keras.layers.Dense(64, activation='elu'),
#     keras.layers.Dense(64, activation='elu'),
#     keras.layers.Dense(64, activation='elu'),    
#     keras.layers.Dropout(0.05),
#     keras.layers.Dense(1)
# ])
# simple_lstm_model.compile(optimizer='adam', loss='mse')
# # 모델 학습
# simple_lstm_model.fit(sequence, target, epochs=80, batch_size=128, verbose=2,
#     shuffle=False, callbacks = [callbacks], validation_split=0.1)

# Epoch 80/80
# - 1s - loss: 3.1128 - val_loss: 3.5588

In [23]:
# # 간단한 lstm 모델 구축하기 
# simple_lstm_model = keras.models.Sequential([
#     keras.layers.LSTM(64, input_shape=sequence.shape[-2:], activation='tanh', return_sequences=True),
#     keras.layers.LSTM(64),
#     keras.layers.Dense(64, activation='elu'),    
#     keras.layers.Dropout(0.05),
#     keras.layers.Dense(1)
# ])
# simple_lstm_model.compile(optimizer='adam', loss='mse')
# # 모델 학습
# simple_lstm_model.fit(sequence, target, epochs=80, batch_size=128, verbose=2,
#     shuffle=False, callbacks = [callbacks], validation_split=0.1)

# Epoch 79/80
#  - 2s - loss: 2.8924 - val_loss: 3.3730
# Epoch 80/80
#  - 2s - loss: 2.7272 - val_loss: 3.1412


In [24]:
# # 간단한 lstm 모델 구축하기 
# simple_lstm_model = keras.models.Sequential([
#     keras.layers.LSTM(64, input_shape=sequence.shape[-2:], activation='tanh', return_sequences=True),
#     keras.layers.LSTM(64),
#     keras.layers.Dense(64, activation='elu'),    
#     keras.layers.Dropout(0.05),
#     keras.layers.Dense(1)
# ])
# simple_lstm_model.compile(optimizer='adam', loss='mse')
# # 모델 학습
# simple_lstm_model.fit(sequence, target, epochs=80, batch_size=32, verbose=2,
#     shuffle=False, callbacks = [callbacks], validation_split=0.1)

# Epoch 78/80
#  - 4s - loss: 2.0523 - val_loss: 2.3489
# Epoch 79/80
#  - 4s - loss: 2.2302 - val_loss: 2.2888
# Epoch 80/80
#  - 4s - loss: 2.0973 - val_loss: 3.1370

In [25]:
# # 간단한 lstm 모델 구축하기 
# simple_lstm_model = keras.models.Sequential([
#     keras.layers.LSTM(256, input_shape=sequence.shape[-2:], activation='tanh'),
# #    keras.layers.LSTM(64, activation='tanh'),
#     keras.layers.Dense(256, activation='elu'),
#     keras.layers.Dropout(0.2),
#     keras.layers.Dense(1)
# ])
# simple_lstm_model.compile(optimizer=Adam(lr=1.e-4), loss='mse')
# # 모델 학습
# simple_lstm_model.fit(sequence, target, epochs=80, batch_size=256, verbose=2,
#     shuffle=False, callbacks = [callbacks], validation_split=0.1)

In [26]:
# LSTM 레이어는 고정

#simple_lstm_model.layers[0].trainable = False

In [27]:
# fine tuning 할 때 사용할 학습데이터 생성 (Y18)
finetune_X, finetune_y = convert_to_timeseries(pd.concat([x_train.tail(432), train['Y18'].tail(432)], axis = 1), interval=n_inte)

100%|██████████████████████████████████████████████████████████████████████████████| 408/408 [00:00<00:00, 1262.52it/s]


In [28]:
finetune_X.shape

(408, 24, 40)

In [29]:
# LSTM 레이어는 고정 시켜두고, DNN 레이어에 대해서 fine tuning 진행 (Transfer Learning)
finetune_history = simple_lstm_model.fit( finetune_X, finetune_y, epochs=30,
            batch_size=20, shuffle=False, verbose = 2, validation_split=0.02)

Train on 399 samples, validate on 9 samples
Epoch 1/30
 - 0s - loss: 6.6506 - val_loss: 4.6411
Epoch 2/30
 - 0s - loss: 6.7680 - val_loss: 5.8535
Epoch 3/30
 - 0s - loss: 3.7948 - val_loss: 3.1142
Epoch 4/30
 - 0s - loss: 3.4055 - val_loss: 2.4223
Epoch 5/30
 - 0s - loss: 3.0712 - val_loss: 2.1342
Epoch 6/30
 - 0s - loss: 2.8509 - val_loss: 2.4088
Epoch 7/30
 - 0s - loss: 2.4544 - val_loss: 1.9798
Epoch 8/30
 - 0s - loss: 2.3041 - val_loss: 2.0394
Epoch 9/30
 - 0s - loss: 2.0567 - val_loss: 1.9031
Epoch 10/30
 - 0s - loss: 1.8873 - val_loss: 1.9321
Epoch 11/30
 - 0s - loss: 1.7043 - val_loss: 1.9338
Epoch 12/30
 - 0s - loss: 1.5527 - val_loss: 1.9821
Epoch 13/30
 - 0s - loss: 1.4039 - val_loss: 2.0617
Epoch 14/30
 - 0s - loss: 1.2784 - val_loss: 2.2193
Epoch 15/30
 - 0s - loss: 1.1702 - val_loss: 2.3323
Epoch 16/30
 - 0s - loss: 1.0705 - val_loss: 2.2881
Epoch 17/30
 - 0s - loss: 1.0205 - val_loss: 2.3740
Epoch 18/30
 - 0s - loss: 0.9485 - val_loss: 2.2672
Epoch 19/30
 - 0s - loss: 0.9

In [30]:
# 예측하기 
finetune_pred = simple_lstm_model.predict(x_test)

In [31]:
# 제출 파일 만들기
submit = pd.DataFrame({'id':range(144*33, 144*113),
              'Y18':finetune_pred.reshape(1,-1)[0]})

In [32]:
submit.to_csv('baseline_result.csv', index = False)